In [1]:
import json
import os 
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from rag_fine_tuning import convert_knowledge_base_to_langchain_docs, fine_tune_rag
from data_utils import convert_json_to_dataframe, create_json_subset, collect_all_results, merge_results
sns.set_style("whitegrid")
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
%load_ext autoreload

# Useful material 
# SQuAD Evaluation guidelines: 
# https://worksheets.codalab.org/worksheets/0x8212d84ca41c4150b555a075b19ccc05/
# https://rajpurkar.github.io/SQuAD-explorer/

# Convert json data to pandas dataframe 

In [ ]:
convert_json_to_dataframe()

In [ ]:
df_all_data = pd.read_csv("dataset.csv")
df_all_data.shape

In [ ]:
df_selected = df_all_data.copy()

In [ ]:
# some rough statistics for the context length 
df_selected.loc[:, "context_chars"] = df_selected["context"].apply(lambda x: len(x))
df_selected.loc[:, "context_words"] = df_selected.loc[:, "context"].apply(lambda x: len(x.split(" ")))

In [ ]:
df_selected.head(2)

In [ ]:
plt.figure(figsize=[8, 5])
sns.histplot(df_selected.drop_duplicates(subset="context")["context_chars"])

In [ ]:
plt.figure(figsize=[8, 5])
sns.histplot(df_selected.drop_duplicates(subset="context")["context_words"])

In [ ]:
# create original json structure for only a subset of questions, used for tests and fine-tuning 
# this file will be used by the evaluation.py file 

df = pd.read_csv("dataset.csv")
df_sel = df[0:500]
df_sel.head(2)

create_json_subset(df_sel)

# RAG architecture

Steps:

**Data Indexing**

Converting text data into a searchable database of vector embeddings, which represent the meaning of the text in a format that computers can easily understand.
- **Documents Chunking**: The collection of documents is split into smaller chunks of text. This allows for more precise and relevant pieces of information to be fed into the language model when needed, avoiding information overload.
- **Vector Embeddings**: The chunks of text are then transformed into vector embeddings. These embeddings encode the meaning of natural language text into numerical representations.
- **Vector Database**: Finally, the vector embeddings are stored in a vector database, making them easily searchable.

**Documents -> Text chunks -> Vector Embeddings -> Vector DB**

**Load -> Split -> Embed -> Store**

## Convert the pandas context to Langchain documents 

In [2]:
df = pd.read_csv("dataset.csv")

langchain_docs = convert_knowledge_base_to_langchain_docs(df)

In [3]:
print(len(langchain_docs))
print(langchain_docs[0])
print(langchain_docs[1])

1204
page_content='The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.' metadata={'title': 'Normans'}
page_content='The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christ

## Vector database

In [4]:
from rag_fine_tuning import CustomRAG, prompt_message, convert_knowledge_base_to_langchain_docs

parameters_dict = {
    "chunk_size": 400,
    "chunk_overlap": 15,
    "vector_database": "chromadb",
    "embeddings_function": {
        "model_name": "text-embedding-3-large",    
        "platform": "OpenAI"
        }, 
    "llm": {
        "model_name": "gpt-3.5-turbo",
        "client": "OpenAI"
        }
}

df_to_test = df[0:2]

rag = CustomRAG(knowledge_base=langchain_docs, 
                prompt_message=prompt_message,
                config=parameters_dict, 
                results_folder='/Users/mariadancianu/Desktop/Git Projects/SQuAD_RAG_experiments/eval_results/test_new_class', 
                vector_db_folder='/Users/mariadancianu/Desktop/Git Projects/SQuAD_RAG_experiments/vector_databases/test_new_class')

("CustomRAG config: {'chunk_size': 400, 'chunk_overlap': 15, "
 "'vector_database': 'chromadb', 'embeddings_function': {'model_name': "
 "'text-embedding-3-large', 'platform': 'OpenAI'}, 'llm': {'model_name': "
 "'gpt-3.5-turbo', 'client': 'OpenAI'}}")


In [5]:
rag.initialize_embeddings_function()

In [6]:
embeddings_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embeddings_model)

db_dir = os.path.join(os.getcwd(), "vector_databases")

rag.create_vector_database()

Vector store 400_text-embedding-3-large already exists. No need to initialize.


## Querying the vector database 

In [7]:
query = "How is the weather today in Milan?"
relevant_docs = rag.query_vector_store(query, n_results=3, score_threshold=0.1)

print(relevant_docs)

Querying chroma vector store
How is the weather today in Milan?


/Users/mariadancianu/Desktop/Git Projects/SQuAD_RAG_experiments/rag_fine_tuning.py:263: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
No relevant docs were retrieved using the relevance score threshold 0.1


[]


In [8]:
query = "Who were the normans?"
relevant_docs = rag.query_vector_store(query, n_results=3, score_threshold=0.1)

print(len(relevant_docs))

for doc in relevant_docs:
    print(doc.page_content)

Querying chroma vector store
Who were the normans?


No relevant docs were retrieved using the relevance score threshold 0.1


0


## Run the RAG over a subset of questions and save the answers 

In [13]:
df_to_test = pd.read_csv("dataset.csv")
df_to_test = df_to_test[0:5]

In [14]:

rag.get_llm_multiple_questions_answers(df_to_test)

Vector store 400_text-embedding-3-large already exists. No need to initialize.


  0%|          | 0/5 [00:00<?, ?it/s]

Querying chroma vector store
In what country is Normandy located?


No relevant docs were retrieved using the relevance score threshold 0.1


** Context: 


...

 80%|████████  | 4/5 [00:07<00:01,  1.80s/it]

Querying chroma vector store
What century did the Normans first gain their separate identity?


No relevant docs were retrieved using the relevance score threshold 0.1


** Context: 


100%|██████████| 5/5 [00:08<00:00,  1.71s/it]


## RAG Fine-tuning 

Comments
Peak distribution for number of characters in the documents is ~600 words. 
- smaller chunks: reduced noise from irrelevant content - works well with dense embeddings*
- larger chunks: preserves context better; ideal if queries require full document context, works well with hybrid search*

TEXT CHUNKING 

1. CHARACTER SPLITTING : divide the text into N-character sized chunks. Can split words in the middle. 
2. RECURSIVE CHARACTER SPLITTING: preserves sentences. Avoids splitting sentences midword (note that RecursiveCharacterTextSplitter with separator does exactly that). Split the
document where a double new line is present, then, if the chunk size is still exceeded, split at new lines, and so on.
3. SEMANTIC SPLITTING: keeps related content together. Use embeddings to split based on meaning.
+ other techniques

EMBEDDINGS 
Create fixed-length vector representation of text, focusing on semanting meaning for tasks like similarity comparison. 
Most up to date embedding models, both proprietary and open source, with performance metrics across different tasks: https://huggingface.co/spaces/mteb/leaderboard 
This contains also a "retrieval" column with performance metrics. Click on the column to sort the models.
Interesting article: https://www.mongodb.com/developer/products/atlas/choose-embedding-model-rag/


In [ ]:
df = pd.read_csv("dataset.csv")
df_to_test = df[0:500]

langchain_docs = convert_knowledge_base_to_langchain_docs(df)

fine_tune_rag(df_to_test, 
              langchain_docs, 
              results_fodler="eval_results/test_new_class", 
              vector_db_folder="vector_databases/test_new_class",)

In [ ]:
df_all_res = collect_all_results('/Users/mariadancianu/Desktop/Git Projects/SQuAD_RAG_experiments/eval_results/initial_eval_results')
df_all_res.sort_values(by="HasAns_f1", ascending=False, inplace=True)
df_all_res.to_csv("eval_results/initial_eval_results/df_all_results.csv", index=False)

In [ ]:
df_all_res

# Investigate the results

In [ ]:
# Pick the best results and merge the scores by question id to the original df in order to inspect the errors.
# The idea is to understand why the results are so poor for the NoAns questions, when the HasAns questions have 
# a high f1 score, in order to understand how the workflow can be optimized

In [ ]:
best_result_path = os.path.join(os.getcwd(), "eval_results/initial_eval_results", df_all_res.experiment.iloc[0])
split_path = best_result_path.split("/")
split_path[-1] = split_path[-1].replace("eval_", "")
best_result_path = "/".join(split_path)
best_result_path

In [ ]:
# run the RAG with best parameters, and save also the context
parameters_dict = {
    "chunk_sizes": [400],
    "embed_options": { 
        "text-embedding-3-large": "OpenAI", 
        },
    "models": {"gpt-3.5-turbo": "openai"}
}
results_folder = os.path.join(os.getcwd(), "eval_results/debugging_eval_results")
results_folder

if not os.path.exists(results_folder):
    os.mkdir(results_folder)

df = pd.read_csv("dataset.csv")
df_to_test = df[0:500]

langchain_docs = convert_knowledge_base_to_langchain_docs(df)

fine_tune_rag(df_to_test, langchain_docs, parameters_dict, results_folder=results_folder, save_context=True)

In [ ]:
! python eval_results/evaluation.py "/Users/mariadancianu/Desktop/Git Projects/SQuAD_RAG_experiments/eval_results/data_updated_500.json" "/Users/mariadancianu/Desktop/Git Projects/SQuAD_RAG_experiments/eval_results/debugging_eval_results/pred_500_400_text-embedding-3-large_gpt-3.5-turbo.json"

In [ ]:
df_merged = merge_results(f1_filepath=os.path.join(os.getcwd(), "eval_results/debugging_eval_results/f1_thresh_by_qid.json"), 
                          exact_filepath=os.path.join(os.getcwd(), "eval_results/debugging_eval_results/exact_thresh_by_qid.json"), 
                          pred_filepath=os.path.join(os.getcwd(), "eval_results/debugging_eval_results/pred_500_400_text-embedding-3-large_gpt-3.5-turbo.json"), 
                          filepath_500=os.path.join(os.getcwd(), "eval_results/debugging_eval_results/pred_500_400_text-embedding-3-large_gpt-3.5-turbo.json"),
                          context_filepath=os.path.join(os.getcwd(), "eval_results/debugging_eval_results/context_500_400_text-embedding-3-large_gpt-3.5-turbo.json"), 
                          df_questions_filepath="dataset.csv", 
                          filter_500=True)

df_merged.shape

In [ ]:
df_merged.columns

In [ ]:
df_merged[df_merged.is_impossible][["id", "is_impossible", "f1_score", "exact_score", "question", "pred"]].tail(10)

In [ ]:
print(df_merged.loc[479, "context"].replace(". ", ".\n"))

In [ ]:
print(df_merged.loc[479, "rag_retrieved_context"])

# Evaluate with other LLMS

In [ ]:
# run the RAG with best parameters, and save also the context
parameters_dict = {
    "chunk_sizes": [400],
    "embed_options": { 
        "text-embedding-3-large": "OpenAI", 
        },
    "models": {"mistral-large-latest": "mistral"}
}
results_folder = os.path.join(os.getcwd(), "eval_results/optimize_results")
results_folder

if not os.path.exists(results_folder):
    os.mkdir(results_folder)

df = pd.read_csv("dataset.csv")
df_to_test = df[0:500]

langchain_docs = convert_knowledge_base_to_langchain_docs(df)

fine_tune_rag(df_to_test, langchain_docs, parameters_dict, results_folder=results_folder, save_context=True)

# Evaluate RAG SOTA embeddings: snowflake-artic-embed-l-v2.0

In [ ]:
# run the RAG with SOTA embeddings 
parameters_dict = {
    "chunk_sizes": [400],
    "embed_options": { 
        "Snowflake/snowflake-arctic-embed-l-v2.0": "HuggingFace_SentenceTransformers" # ranked 6th, 568M params, released in december 2024 
        },
    "models": {"gpt-3.5-turbo": "openai"}
}

results_folder = os.path.join(os.getcwd(), "eval_results/optimize_results")
results_folder

if not os.path.exists(results_folder):
    os.mkdir(results_folder)

In [ ]:
df = pd.read_csv("dataset.csv")
df_to_test = df[0:500]

langchain_docs = convert_knowledge_base_to_langchain_docs(df)

fine_tune_rag(df_to_test, langchain_docs, parameters_dict, results_folder=results_folder, save_context=False)